This notebook shows the results of the torch to tf translation function for 3 models:
- a CNN with Dropout
- the same CNN, but with intermediate layer output (to get better insights)
- the same CNN, but without dropout

Results:
- the CNN with dropout has diverging outputs in torch and tf
- without dropout, the outputs are the same

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [5]:
## get intermediate layers of pytorch model

class LayerNet(Net):

    def __init__(self):
        super().__init__()

    def forward(self, x):
        x1 = F.relu(F.max_pool2d(self.conv1(x), 2))
        x2 = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x1)), 2))
        x3 = x2.view(-1, 320)
        x4 = F.relu(self.fc1(x3))
        x5 = F.dropout(x4, training=self.training)
        x6 = self.fc2(x5)
        return [x1, x2, x3, x4, x5, x6, F.log_softmax(x6, dim=1)]

In [30]:
class NoDropoutNet(nn.Module):
    def __init__(self):
        super(NoDropoutNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [32]:
import argparse
import torch.optim as optim
from torchvision import datasets, transforms

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')

# Train this model with 60 epochs and after process every 300 batches log the train status
args = parser.parse_args(['--epochs', '10', '--log-interval', '300'])

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


model = NoDropoutNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)



In [33]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.342345
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.268953
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.295525
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.073861

Test set: Average loss: 0.1234, Accuracy: 9636/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.166110
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.160232
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.063766
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.174566

Test set: Average loss: 0.0768, Accuracy: 9772/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.013016
Train Epoch: 3 [19200/60000 (32%)]	Loss: 0.216341
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.111469
Train Epoch: 3 [57600/60000 (96%)]	Loss: 0.075249

Test set: Average loss: 0.0611, Accuracy: 9802/10000 (98%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.192573
Train Epoch: 4 [19200/60000 (32%)]	Loss: 0.074138
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.061015
Train Epoch: 4 [57600/60000 (96%)]	Loss: 0.013557

Test set: Average loss: 0.0553, Accu

In [34]:
# Save the trained model to a file
torch.save(model.state_dict(), 'models/mnist_pytorch_24_06_22_no_dropout.pth')

In [35]:
from Main import translate
tf_rep = translate(model, 'tf2', dummy_input=torch.randn([64, 1, 28, 28]))

INFO:tensorflow:Assets written to: models/mnist_tf.pb\assets


INFO:tensorflow:Assets written to: models/mnist_tf.pb\assets


In [18]:
import tensorflow as tf
print(tf_rep.signatures)
print(tf_rep.signatures['serving_default'])
tf_layers = tf.keras.Model(inputs=tf_rep.inputs, outputs=[tf_rep.layers[1].output]) + tf_rep.outputs

_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, input.1) at 0x1D7C866E100>})
ConcreteFunction signature_wrapper(*, input.1)
  Args:
    input.1: float32 Tensor, shape=(64, 1, 28, 28)
  Returns:
    {'20': <1>, 'input': <2>, 'onnx::Gemm_16': <3>, 'onnx::Reshape_14': <4>, 'x4': <5>, 'x6': <6>}
      <1>: float32 Tensor, shape=(64, 10)
      <2>: float32 Tensor, shape=(64, 10, 12, 12)
      <3>: float32 Tensor, shape=(64, 320)
      <4>: float32 Tensor, shape=(64, 20, 4, 4)
      <5>: float32 Tensor, shape=(64, 50)
      <6>: float32 Tensor, shape=(64, 10)


AttributeError: '_UserObject' object has no attribute 'inputs'

In [38]:
import matplotlib.pyplot as plt
import numpy as np

features, labels = next(iter(train_loader))
tf_logits_full = tf_rep(**{'input.1':features})
tf_logits = tf_logits_full['20']
#tf_input_layer = tf_logits_full['input']
tf_output = np.argmax(tf_logits, axis=1)
torch_logits_full = model(features)
torch_logits = torch_logits_full
#torch_input_layer = torch_logits_full[0]
torch_output = torch.argmax(torch_logits, dim=1)
for torch_l, torch_o, tf_l, tf_o, y in zip(torch_logits, torch_output, tf_logits, tf_output, labels.tolist()):
        print(f'torch-output: {torch_o}, tf-output: {tf_o}, label: {y}')
        print(f'torch-logits: {torch_l}')
        print(f'tf-logits: {tf_l}')
        # print(f'torch-input-layer: {torch_i[0]}')
        # print(f'tf-input-layer: {tf_i[0]}')

torch-output: 3, tf-output: 3, label: 3
torch-logits: tensor([-2.2453e+01, -1.8735e+01, -1.6440e+01, -1.4305e-06, -2.0227e+01,
        -1.4445e+01, -2.3794e+01, -2.2566e+01, -1.4096e+01, -1.5670e+01],
       grad_fn=<UnbindBackward0>)
tf-logits: [-2.2453272e+01 -1.8735226e+01 -1.6440443e+01 -1.6689287e-06
 -2.0227110e+01 -1.4444656e+01 -2.3794052e+01 -2.2565966e+01
 -1.4095890e+01 -1.5670005e+01]
torch-output: 1, tf-output: 1, label: 1
torch-logits: tensor([-1.2540e+01, -3.1590e-05, -1.2594e+01, -2.2395e+01, -1.0794e+01,
        -1.9026e+01, -1.7555e+01, -1.2982e+01, -1.3298e+01, -1.5403e+01],
       grad_fn=<UnbindBackward0>)
tf-logits: [-1.2540019e+01 -3.1709169e-05 -1.2594376e+01 -2.2394575e+01
 -1.0794280e+01 -1.9025883e+01 -1.7555111e+01 -1.2982286e+01
 -1.3298212e+01 -1.5403474e+01]
torch-output: 3, tf-output: 3, label: 3
torch-logits: tensor([-2.1479e+01, -1.7841e+01, -1.6366e+01, -2.7436e-03, -1.9129e+01,
        -1.1551e+01, -3.0253e+01, -1.8565e+01, -9.9491e+00, -5.9211e+00],